In [16]:
%%writefile swiss_cheese.py

import os

def create_base_directory(geometry):
    root = os.getcwd()
    geometry_dir = os.path.join(root, geometry)
    if not os.path.isdir(geometry_dir):
        os.mkdir(geometry_dir)
        return geometry_dir
    else:
        return geometry_dir

def create_holes_directories(base, N_roles):
    N_roles_dir_name = '{}_roles'.format(N_roles)
    N_roles_subdir = os.path.join(base, N_roles_dir_name)
    if not os.path.isdir(N_roles_subdir):
        os.mkdir(N_roles_subdir)
    return N_roles_subdir  

def create_temperature_subdirectory(N_roles_subdir, T):
    T_subdirs = []    
    T_subdir = os.path.join(N_roles_subdir, 'T{}'.format(T))
    if not os.path.isdir(T_subdir):
        os.mkdir(T_subdir)
    T_subdirs.append(T_subdir)
    return T_subdirs

def create_temperature_subdirectories(T_subdirs):
    energylog_dirs = []
    hysteresis_dirs = []    
    groundstates_dirs = []
    scripts_dirs = []
    plots_dirs = []
    # subdirs
    sub_dirs = ['cubit',
                'energylog',
                'final_path',
                'groundstates',
                'hysteresis',
                'initial_path',
                'NEB',
                'patran',
                'plots',
                'scripts']
    # for each elongation, creates the subdirs
    for T_subdir in T_subdirs:
        for sub_dir in sub_dirs:
            new_sub_dir = os.path.join(T_subdir,sub_dir)
            if not os.path.isdir(new_sub_dir):
                os.mkdir(new_sub_dir)
            # energgylog
            if sub_dir == 'energylog':
                energylog_dirs.append(new_sub_dir)
            # hysteresis
            elif sub_dir == 'hysteresis':
                hysteresis_dirs.append(new_sub_dir)
            # scripts
            elif sub_dir == 'scripts':
                scripts_dirs.append(new_sub_dir)
            # groundstates
            elif sub_dir == 'groundstates':
                groundstates_dirs.append(new_sub_dir)
            # plots
            elif sub_dir == 'plots':
                plots_dirs.append(new_sub_dir)
    return [energylog_dirs, 
            hysteresis_dirs, 
            groundstates_dirs,
            scripts_dirs,
            plots_dirs]
            
def create_energylog_subdirs(energylog_dirs, sizes):
    subdirs, groundstates_subdirs = [], []
    energylog_sub_dirs = ['groundstates','energy_barriers']
    for energylog_dir in energylog_dirs:
        for energylog_sub_dir in energylog_sub_dirs:
            path = os.path.join(energylog_dir, energylog_sub_dir)
            if not os.path.isdir(path):
                os.mkdir(path)
            if energylog_sub_dir == 'groundstates':
                for size in sizes:
                    size_path = os.path.join(path, str(size))
                    if not os.path.isdir(size_path):
                        os.mkdir(size_path)
                    groundstates_subdirs.append(size_path)
            subdirs.append(path)
    
def create_hysteresis_subdirs(hysteresis_dirs, sizes):
    groundstates_subdirs, loops_subdirs, size_hyst_subdirs = [], [], []
    hysteresis_sub_dirs = ['groundstates','loops', 'size_loop']
    for hysteresis_dir in hysteresis_dirs:
        for hysteresis_sub_dir in hysteresis_sub_dirs:
            path = os.path.join(hysteresis_dir, hysteresis_sub_dir)
            if not os.path.isdir(path):
                os.mkdir(path)
            if hysteresis_sub_dir == 'groundstates':
                for size in sizes:
                    size_path = os.path.join(path, str(size))
                    if not os.path.isdir(size_path):
                        os.mkdir(size_path)
                    groundstates_subdirs.append(size_path)
            if hysteresis_sub_dir == 'loops':
                for size in sizes:
                    loops_subdir = os.path.join(path, str(size))
                    if not os.path.isdir(loops_subdir):
                        os.mkdir(loops_subdir)
                    loops_subdirs.append(loops_subdir)
            if hysteresis_sub_dir == 'size_loop':
                for sub in ['states','hyst']:
                    sub_path = os.path.join(path, sub)
                    if not os.path.isdir(sub_path):
                        os.mkdir(sub_path)
                    size_hyst_subdirs.append(size_path)
            else:
                loops_subdirs.append(path)         
                
def create_scripts_subdirs(scripts_dirs, sizes):
    scripts_sub_dirs = ['groundstates','energy_barrier', 'batch', 'hysteresis', 'size_hysteresis']
    for scripts_dir in scripts_dirs:
        for scripts_sub_dir in scripts_sub_dirs:
            path = os.path.join(scripts_dir, scripts_sub_dir)
            if not os.path.isdir(path):
                os.mkdir(path)
            if scripts_sub_dir == 'hysteresis':
                for size in sizes:
                    ss = os.path.join(path, str(size))
                    if not os.path.isdir(ss):
                        os.mkdir(ss)
        
def create_groundstates_subdirs(groundstates_dirs, sizes): 
    for groundstates_dir in groundstates_dirs:
        for size in sizes:
            path = os.path.join(groundstates_dir, str(size))
            if not os.path.isdir(path):
                os.mkdir(path)           
                        
def create_all_directories(geometry, N_roles, Ts, sizes):
    base = create_base_directory(geometry)
    for N_role in N_roles:
        for T in Ts:
            hole_dir = create_holes_directories(base, N_role)
            temp_dir = create_temperature_subdirectory(hole_dir, T)
            temp_subdirs = create_temperature_subdirectories(temp_dir)
            # sub directories
            energylog_dirs = temp_subdirs[0]
            hysteresis_dirs = temp_subdirs[1]
            groundstates_dirs = temp_subdirs[2]
            scripts_dirs = temp_subdirs[3]
            plots_dirs = temp_subdirs[4]
            # sub sub directories
            create_energylog_subdirs(energylog_dirs, sizes)
            create_hysteresis_subdirs(hysteresis_dirs, sizes)
            create_scripts_subdirs(scripts_dirs, sizes)
            create_groundstates_subdirs(groundstates_dirs, sizes)


Overwriting swiss_cheese.py


In [17]:
def get_random_position(size, R_holes): 
    r = 2 * (np.random.rand(3) - 0.5)
    while (r[0]**2 + r[1]**2 + r[2]**2) > 1:
        r = 2 * (np.random.rand(3) - 0.5)
    return r * (size - R_holes) 

def create_swiss_cheese_mesh(size, R_holes, N_holes):
    meshsize = 0.009
    s = "brick x {0:g} \n".format(R_particle)
    n_swiss_cheese = 1
    n_hole = 2
    for n in range(N_holes):
        s += "create sphere radius {0:g} \n".format(R_holes)
        r = get_random_position(R_particle, R_holes)
        s += "volume {0:g} move x {1:g} y {2:g} z {3:g}  \n".format(n_hole, r[0], r[1], r[2])
        s += "subtract volume {0:g} from volume {1:g} \n".format(n_hole, n_swiss_cheese)
        n_swiss_cheese = n_hole + 1
        n_hole = n_swiss_cheese + 1
    s += "volume {0:g} size {1:g}\n".format(n_swiss_cheese, meshsize)
    s += "volume {0:g} scheme Tetmesh \n".format(n_swiss_cheese)
    s += "mesh volume {0:g} \n".format(n_swiss_cheese)
    s += "block 1 volume {0:g} \n".format(n_swiss_cheese)
    s += "block 1 element type tetra4\n".format()
    s += "set patran export groups on\n".format()
    s += 'export patran "{0:g}_{1:g}_{2:g}.pat"\n'.format(R_particle, R_holes*1000, N_holes)
    return s
            
def create_swiss_cheese_script(R_particle, R_holes, N_holes):
    s = "magnetite 20 C \n"
    s += "ReadMesh 1 {0:g}_{1:g}_{2:g}.pat \n".format(R_particle, R_holes*1000, N_holes)
    s += "Randomize all moments \n"
    s += "external field direction 1 0 0 \n"
    s += "external field strength 0.000000 mT \n"
    s += "Minimize \n"
    s += "WriteMagnetization output/{0:g}_{1:g}_{2:g} \n".format(R_particle, R_holes*1000, N_holes)
    s += "WriteHyst output/0.5_30_50.hyst \n".format(R_particle, R_holes*1000, N_holes)
    return s    
    
def create_cubit_scripts(N_holes, T, size):
    base = create_holes_directories(create_base_directory(geometry), N_roles)
    cubit_file_path = os.path.join(base,
                                  'T{}'.format(T),
                                  'cubit')
    if N_holes == 0:
        pass
    else:
        pass